# Konlpy 사용하여 전처리

In [7]:
!pip install konlpy
import time
import re
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

courses_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/merged_lecture.csv')
crawling_df = pd.read_csv('/content/drive/MyDrive/PROJECT/교육데이터분석공모전/데이터분석/모델링_예측/크롤링 데이터/크롤링_전처리.csv')

In [8]:

# 텍스트 전처리 함수
okt = Okt()



def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    # 텍스트 정규화
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)  # 특수문자 제거
    text = okt.morphs(text)  # 형태소 분석

    return text  # 예시로 단순히 반환하는 코드
    return ' '.join(text)

# 전처리 시간 측정
start_time = time.time()
courses_df['강의 명'] = courses_df['강의 명'].apply(preprocess_text)
crawling_df['제목'] = crawling_df['제목'].apply(preprocess_text)
crawling_df['설명'] = crawling_df['설명'].apply(preprocess_text)
preprocessing_time = time.time() - start_time
print(f"전처리 시간: {preprocessing_time:.2f}초")



전처리 시간: 810.26초


In [10]:
import numpy as np
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()

# 강의 명과 설명을 하나의 텍스트로 결합
crawling_df['전체 텍스트'] = crawling_df['제목'] + " " + crawling_df['설명']
# NaN 값 제거
crawling_df['전체 텍스트'] = crawling_df['전체 텍스트'].fillna('')
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix_courses = vectorizer.fit_transform(courses_df['강의 명'])
tfidf_matrix_crawling = vectorizer.transform(crawling_df['전체 텍스트'])


TypeError: can only concatenate list (not "str") to list

In [ ]:
# 관심사 입력받기
interests = ["에듀테크", "정원"]

# 관심사 임베딩 생성
interest_embeddings = vectorizer.transform(interests)

# 맞춤형 강의 추천 함수
def recommend_courses(interest_embeddings, tfidf_matrix_courses, course_titles):
    similarities = cosine_similarity(interest_embeddings, tfidf_matrix_courses)
    recommended_courses = []
    for sim in similarities:
        recommended_courses.append(course_titles[np.argmax(sim)])
    return recommended_courses

# 추천 강의명 출력
recommended_courses = recommend_courses(interest_embeddings, tfidf_matrix_courses, courses_df['강의 명'])

print("추천 강의명:")
for course in recommended_courses:
    print(course)


In [ ]:
# 추천 시간 측정
start_time = time.time()
scores = recommend_courses(tfidf_matrix_courses, interest_embeddings, weights)
courses_df['추천 점수'] = scores
recommendation_time = time.time() - start_time
print(f"추천 시간: {recommendation_time:.2f}초")


In [ ]:
!pip install transformers
import time
import torch
from transformers import BertTokenizer, BertModel


# BERT 모델 로드
start_time = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model_loading_time = time.time() - start_time
print(f"BERT 모델 로딩 시간: {model_loading_time:.2f}초")




In [ ]:

# 관심사 입력 및 가중치 부여 (예시)
interests = ['취미', '건강']
weights = [0.7, 0.3]

# 텍스트 임베딩 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# 임베딩 생성 시간 측정
start_time = time.time()
courses_df['임베딩'] = courses_df['강의 명'].apply(get_embedding)
crawling_df['임베딩'] = (crawling_df['제목'] + " " + crawling_df['설명']).apply(get_embedding)
embedding_time = time.time() - start_time
print(f"임베딩 생성 시간: {embedding_time:.2f}초")

In [ ]:
# 관심사 임베딩 생성
interest_embeddings = [get_embedding(preprocess_text(interest)) for interest in interests]

# 맞춤형 강의 추천
def recommend_courses(course_embeddings, interest_embeddings, weights):
    recommendations = []
    for course_embedding in course_embeddings:
        score = 0
        for interest_embedding, weight in zip(interest_embeddings, weights):
            score += weight * torch.cosine_similarity(course_embedding.unsqueeze(0), interest_embedding.unsqueeze(0)).item()
        recommendations.append(score)
    return recommendations

In [ ]:

# 추천 점수 기준으로 정렬
recommended_courses = courses_df.sort_values(by='추천 점수', ascending=False)
print(recommended_courses)
